In [1]:
PROJECT_NAME = "3LC Tutorials"
DATASET_NAME = "ADE20k_toy_dataset"
TRANSIENT_DATA_PATH = "../../transient_data"


In [2]:
import tlc
from pathlib import Path

In [3]:
DATASET_ROOT = (Path(TRANSIENT_DATA_PATH) / "ADE20k_toy_dataset").resolve().as_posix()
tlc.register_url_alias("ADE20K_TOY_DATASET_ROOT", DATASET_ROOT)

In [4]:
table = tlc.Table.from_names("train", DATASET_NAME, PROJECT_NAME)

In [5]:
table[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=683x512>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=683x512>,
 (512, 683))

In [6]:
table.table_rows[0]

ImmutableDict({'image': '<ADE20K_TOY_DATASET_ROOT>/images/training/ADE_train_00000001.jpg', 'segmentation_map': '<ADE20K_TOY_DATASET_ROOT>/annotations/training/ADE_train_00000001.png', 'mask size': {'width': 512, 'height': 683}, 'weight': 1.0})

In [7]:
value_map = table.get_value_map("segmentation_map")
print(value_map)

3lc: Could not get value map for value path segmentation_map: The schema for the path must be numeric.
None


In [27]:
table_writer = tlc.TableWriter(
    table_name=f"{table.name}_instances",
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    column_schemas={
        "image": tlc.PILImage("image"),
        "instances": tlc.InstanceSegmentationMasks(
            "instances",
            instance_properties_structure={
                "label": tlc.CategoricalLabel("label", value_map),
            },
        ),
    },
    if_exists="rename",
)

In [ ]:
segmentation_image = table.table_rows[0]    

In [20]:
value_map = table.rows_schema.values["segmentation_map"].value.map


In [9]:
table.columns


['image', 'segmentation_map', 'mask size', 'weight']

In [11]:
import numpy as np

In [32]:
def single_channel_map_to_per_class_masks(map: np.ndarray):
    masks = []
    class_ids = []
    for class_id in np.unique(map):
        mask = (map == class_id).astype(np.uint8)
        masks.append(mask)
        class_ids.append(class_id)
    return np.asfortranarray(np.stack(masks, axis=-1)), np.array(class_ids)


In [33]:
for image, map, (h, w) in table:
    map_np = np.array(map)
    masks, class_ids = single_channel_map_to_per_class_masks(map_np)
    print(masks.shape)
    print(class_ids)

    table_writer.add_row(
        {
            "image": image,
            "instances": {
                "image_height": h,
                "image_width": w,
                "masks": masks,
                "instance_properties": {
                    "label": class_ids.tolist(),
                },
            },
        }
    )



(512, 683, 17)
[  0   1   4   5   6  13  18  32  33  43  44  88  97 105 126 139 150]


In [34]:
table = table_writer.finalize()